# 자연어처리 시작하기 - 전처리, 토큰화, 명사추출

In [ ]:
!mkdir hanpre

In [ ]:
%%writefile hanpre/__init__.py

from . import funcs
from . import stopwords

Writing hanpre/__init__.py


In [ ]:
!ls hanpre

__init__.py


In [ ]:
%%writefile hanpre/funcs.py

import sys
import csv
import itertools
import pandas as pd
import numpy as np
import re
from csv import DictReader
from datetime import datetime as dt
from pandas import read_excel

from hanpre import stopwords as sw

maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

        
##################################################################################################################################
def save_to_csv(ftype, keyword, df):
    # save to csv    
    filename = "./input/" + ftype + "_" + keyword.replace(" ","") + ".csv"   
    # filename_list.append(filename)
    df.to_csv(filename, date_format='%Y%m%d', encoding='utf-8-sig')

def interested_words(keywords):
    # 핵심단어 읽어 오기
    my_sheet = '소비키워드'
    keywords_filename = 'deskresearch__.xlsx'
    df = read_excel(keywords_filename, sheet_name = my_sheet, header=0) # index_col='번호'

    interested_words = df
#     tdf = df.T
#     keywords = tdf[0].tolist()
#     sunkeywords = tdf[1].tolist()

    iwlist = []
    for i in range(1,len(keywords)+1):
        iwlist.append(interested_words[i].dropna().tolist())
        
    return iwlist # keywords, sunkeywords, 

def preprocessing(text):
    # 불필요 Text 정리
    text = re.sub('\\\\n', ' ', text)
    text = re.sub('\[a-z]+[a-z]$', '', text)
    text = re.sub('\t', '', text)
    text = re.sub('\.+', '', text)
    text = re.sub('[\:|!|-|_|✦|@|,]', ' ', text)
    text = re.sub('[0-9]', '', text)
    text = re.sub('[a-z]', '', text)
    text = re.sub('[\+|《|》|ᕕ|-|-|%|?|&]', '', text)
    text = re.sub(',+', ',', text)
    text = re.sub('\.+', '.', text)
    text = re.sub('[\'\"]', '', text)
    text = re.sub('[\'\n"]', '', text)
    text = re.sub('\s+|\t|\n', ' ', text).strip()
    text = re.sub('ㅇ+|ㅎ+|ㅋ+|ㅠ+|\^+|ㅜ+|▼+|▶+|\*+|<+|☆+|♥+|🥝+|ㄱ+|~+|#+|;+|♡|★|●|•|】|❤️|▲|♬|☎|◆|✊|◈|♠|■|☻|◀|🅴|囧', '', text)
    text = re.sub('♣|>+|✔|✅| ૢ|❛|◡|❛|✿', ' ', text)
    text = re.sub('[-+]', '', text)
    text = re.sub('^[0-9].*[0-9]$', '', text)
    text = re.sub('^[0-9]+[년|월|일|시|g]$', '', text)
    text = re.sub('^[0-9]+[원|천원|만원|백만원|억|억원]$', '', text)
    text = re.sub('^[0-9]+[명|점|개|종|회|주|층|등|배|번|차|화|위|번]$', '', text)
    text = re.sub('.*(광고정보).*', '', text)
    text = re.sub('.*(기사출처).*', '', text)
    text = re.sub('[\'\"]', '', text)
    text = re.sub('[\'\n"]', '', text)
    text = re.sub('^[http|naver|com|daum|www].*', '', text)
    text = re.sub('/|『|』|‘|\[|\]|\(|\)', ' ', text)
#     text = ''.join([i for i in text if not i.isdigit()])
    text = text.strip()        
    return text

def oneDArray(x):
    return list(itertools.chain(*x))

def remove_stopwords(lst):
    result = []
    for text in lst:
        text = preprocessing(text)
        if ((len(text) > 1) and (not text.isdigit())):
            if text not in sw.stopwords_kr: 
                result.append(text)
    return result

def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

    
def read_naver_blog(keyword):

    path = './webscraping/scraps'
    csvfile = path + f'/1_naver_blog/naver-blog-contents_{keyword}.csv'

    df = pd.read_csv(csvfile)
    df = df[['title','content','date']]
    
    df['title'] = df['title'].apply(lambda x: re.sub(r"[\n\t?]", '', str(x)))
    df['title'] = df['title'].apply(lambda x: re.sub(r"\.+$", '', str(x)))
    df['title'] = df['title'].apply(lambda x: re.sub(r"\[질문\]", '', str(x)))
    df['date'] = df['date'].apply(lambda x: re.sub(r"^\D(\d{5})\D", '(d{-4})', str(x)))
    df['date'] = df['date'].apply(lambda x: x.replace(".", "-")).apply(lambda x: x.replace(" ", "")).apply(lambda x: x[:-1])
    df['date'] = df['date'].apply(lambda x: re.sub(r'.*시간.*','2020-10-02', str(x)))
    df['date'] = df['date'].apply(lambda x: re.sub(r'.*분.*','2020-10-02', str(x)))
    df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')
    
    return df

def read_naver_kin(keyword):

    path = './webscraping/scraps'
    csvfile = path + f'/2_naver_kin/kin-posts-scrapped_{keyword}.csv'

    df = pd.read_csv(csvfile)
    df = df[['question','answer','date']]
    df.columns = [ 'title', 'content', 'date' ] 
    
    df['title'] = df['title'].apply(lambda x: re.sub(r"[\n\t?]", '', str(x)))
    df['title'] = df['title'].apply(lambda x: re.sub(r"\.+$", '', str(x)))
    df['title'] = df['title'].apply(lambda x: re.sub(r"\[질문\]", '', str(x)))
    df['date'] = df['date'].apply(lambda x: re.sub(r"^\D(\d{5})\D", '(d{-4})', str(x)))
    df['date'] = df['date'].apply(lambda x: x.replace(".", "-")).apply(lambda x: x.replace(" ", "")).apply(lambda x: x[:-1])
    df['date'] = df['date'].apply(lambda x: re.sub(r'.*시간.*','2020-10-02', str(x)))
    df['date'] = df['date'].apply(lambda x: re.sub(r'.*분.*','2020-10-02', str(x)))
    df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')

    return df

def read_naver_cafe(keyword):

    path = './webscraping/scraps'
    csvfile = path + f'/3_naver_cafe/naver-cafe-scrapped_{keyword}.csv'

    df = pd.read_csv(csvfile)
    df = df[['title','content','date']]

    df['title'] = df['title'].apply(lambda x: re.sub(r"[\n\t?]", '', str(x)))
    df['title'] = df['title'].apply(lambda x: re.sub(r"\.+$", '', str(x)))
    df['title'] = df['title'].apply(lambda x: re.sub(r"작성일", '', str(x)))
    df['date'] = df['date'].apply(lambda x: re.sub(r"작성일", '', str(x)))
    df['date'] = df['date'].apply(lambda x: x.split(" ")[0])
    df['date'] = df['date'].apply(lambda x: x.replace(".", "-")).apply(lambda x: x.replace(" ", "")).apply(lambda x: x[:-1])
    df['date'] = df['date'].apply(lambda x: re.sub(r'.*시간.*','2020-10-02', str(x)))
    df['date'] = df['date'].apply(lambda x: re.sub(r'.*분.*','2020-10-02', str(x)))
    df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')

    return df

def read_daum_news(keyword):

    path = './webscraping/scraps'
    csvfile = path + f'/4_daum_news/daum-news-scrapped_{keyword}.csv'

    df = pd.read_csv(csvfile)
    df = df[['title','content','date']]
    
    df['title'] = df['title'].apply(lambda x: re.sub(r"[\n\t?]", '', str(x)))
    df['title'] = df['title'].apply(lambda x: re.sub(r"\.+$", '', str(x)))
    df['title'] = df['title'].apply(lambda x: re.sub(r"\[질문\]", '', str(x)))
    
    df['date'] = df['date'].apply(lambda x: x.split("|")[0])
    df['date'] = df['date'].apply(lambda x: re.sub(r"^\D(\d{5})\D", '(d{-4})', str(x)))
    df['date'] = df['date'].apply(lambda x: x.replace(".", "-")).apply(lambda x: x.replace(" ", "")) # .apply(lambda x: x[:-1])
    df['date'] = df['date'].apply(lambda x: re.sub(r'.*시간.*','2020-10-02', str(x)))
    df['date'] = df['date'].apply(lambda x: re.sub(r'.*분.*','2020-10-02', str(x)))
    df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')
    
    return df

def read_bigkinds(keyword):

    path = './webscraping/scraps'
    csvfile = path + f'/5_bigkinds/bigkinds_{keyword}.csv'

    df = pd.read_csv(csvfile)
    df = df[['title','content','date']]
    
    df['title'] = df['title'].apply(lambda x: re.sub(r"\"", '', str(x)))
    
    df['date'] = df['date'].apply(lambda x: re.sub(r"/", '-', str(x))).apply(lambda x: x.replace(" ", "")) #.apply(lambda x: x[:-1])
    df['date'] = df['date'].apply(lambda x: re.sub(r'^\D+','2020-10-02', str(x)))
    df['date'] = df['date'].apply(lambda x: re.sub(r'.*시간.*','2020-10-02', str(x)))
    df['date'] = df['date'].apply(lambda x: re.sub(r'.*분.*','2020-10-02', str(x)))
    df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')
    
    return df


def read_tweeter(keyword):

    path = './webscraping/scraps'
    csvfile = path + f'/7_tweeter/tweeter-scrapped_{keyword}.csv'

    df = pd.read_csv(csvfile)
    df = df[['title','content','date']]
    
    df['date'] = df['date'].apply(lambda x: re.sub(r"(월|일)", '-', str(x))).apply(lambda x: '2020-' + x).apply(lambda x: x[:-1])
    df['date'] = df['date'].apply(lambda x: re.sub(r" ", '', str(x)))
    df['date'] = df['date'].apply(lambda x: re.sub(r'.*시간.*','2020-10-02', str(x)))
    df['date'] = df['date'].apply(lambda x: re.sub(r'.*분.*','2020-10-02', str(x)))
    df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')
    
    return df


def read_instagram(keyword):

    path = './webscraping/scraps'
    csvfile = path + f'/6_instagram/insta-scrapped_{keyword}.csv'

    df = pd.read_csv(csvfile)
    df = df[['comment','comment','date']]
    df.columns = [ 'title', 'content', 'date' ] 
    
    df['title'] = df['title'].apply(lambda x: re.sub(r"[a-z]+|[A-Z]+|[0-9]+", '', str(x)))
    df['title'] = df['title'].apply(lambda x: re.sub(r"(\\|\.|,|:|\n|@|_|\[|\]|・|🦐|🌸|🦀|🐙)", '', str(x)))
    df['title'] = df['title'].apply(lambda x: x.strip())
    
    df['content'] = df['content'].apply(lambda x: re.sub(r"[a-z]+|[A-Z]+|[0-9]+", '', str(x)))
    df['content'] = df['content'].apply(lambda x: re.sub(r"(\\|\.|,|:|\n|@|_|\[|\]|・|🦐|🌸|🦀|🐙)", '', str(x)))
    df['content'] = df['content'].apply(lambda x: x.strip())
    
    df['date'] = pd.to_datetime(df.date, format='%B %d, %Y', errors='coerce')
#     df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date'])
        
    return df

def readall(keyword):
    df1 = read_naver_blog(keyword)
    df2 = read_naver_kin(keyword)
    df3 = read_naver_cafe(keyword)
    df4 = read_daum_news(keyword)
    df5 = read_bigkinds(keyword)
    df6 = read_tweeter(keyword)
    df7 = read_instagram(keyword)
    df= df1.append([df2,df3,df4,df5,df6,df7])
    
    return df

Writing hanpre/funcs.py


#######################################################################################################################################

In [ ]:
keyword = "외식"

In [ ]:
df6 = read_tweeter(keyword)

df6.head()

NameError: ignored

In [ ]:
def read_instagram(keyword):

    path = './webscraping/scraps'
    csvfile = path + f'/6_instagram/insta-scrapped_{keyword}.csv'

    df = pd.read_csv(csvfile)
    df = df[['comment','comment','date']]
    df.columns = [ 'title', 'content', 'date' ] 
    
    df['title'] = df['title'].apply(lambda x: re.sub(r"[a-z]+|[A-Z]+|[0-9]+", '', str(x)))
    df['title'] = df['title'].apply(lambda x: re.sub(r"(\\|\.|,|:|\n|@|_|\[|\]|・|🦐|🌸|🦀|🐙)", '', str(x)))
    df['title'] = df['title'].apply(lambda x: x.strip())
    
    df['content'] = df['content'].apply(lambda x: re.sub(r"[a-z]+|[A-Z]+|[0-9]+", '', str(x)))
    df['content'] = df['content'].apply(lambda x: re.sub(r"(\\|\.|,|:|\n|@|_|\[|\]|・|🦐|🌸|🦀|🐙)", '', str(x)))
    df['content'] = df['content'].apply(lambda x: x.strip())
    
    df['date'] = pd.to_datetime(df.date, format='%B %d, %Y', errors='coerce')
#     df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date'])
        
    return df

In [ ]:
# keywords = ['외식', '식사', '메뉴', '식당', '레스토랑', '가성비', '트렌드', '배달', '맛집', '비대면']

# for keyword in keywords:
#     df7 = read_instagram(keyword)
#     print(df7[:7])
    

                                  title                               content  \
0                    지닝     사진 설명이 없습니다                    지닝     사진 설명이 없습니다   
1                        다미👋     이미지 음식                        다미👋     이미지 음식   
2                  보경     이미지 테이블 음식                  보경     이미지 테이블 음식   
4       이미지 앉아 있는 사람들 음식을 먹는 사람들 테이블 음식       이미지 앉아 있는 사람들 음식을 먹는 사람들 테이블 음식   
5                 이미지 사람 명 앉아 있는 사람들 음식                 이미지 사람 명 앉아 있는 사람들 음식   
6                           사진 설명이 없습니다                           사진 설명이 없습니다   
7  지나씨     이미지 사람 명 앉아 있는 사람들 어린이 실내 음식  지나씨     이미지 사람 명 앉아 있는 사람들 어린이 실내 음식   

        date  
0 2020-09-19  
1 2020-09-14  
2 2020-09-28  
4 2020-09-26  
5 2020-09-12  
6 2020-08-30  
7 2020-09-10  
                            title                         content       date
0                      이미지 테이블 음식                      이미지 테이블 음식 2020-09-30
1             じょり     이미지 사람 명 음식             じょり     이미지 사람 명 음식 2020-10-01


In [ ]:
df7.head()

,title,content,date
0,지닝 사진 설명이 없습니다,지닝 사진 설명이 없습니다,2020-09-19
1,다미👋 이미지 음식,다미👋 이미지 음식,2020-09-14
2,보경 이미지 테이블 음식,보경 이미지 테이블 음식,2020-09-28
3,조희빈 이미지 실내,조희빈 이미지 실내,NaT
4,이미지 앉아 있는 사람들 음식을 먹는 사람들 테이블 음식,이미지 앉아 있는 사람들 음식을 먹는 사람들 테이블 음식,2020-09-26


In [ ]:
# 공백라인 및 NaN 제거
while("" in sentences) : 
    sentences.remove("") 
while("NaN" in sentences) : 
    sentences.remove("") 

In [ ]:
def readall(keyword):
    df1 = read_naver_blog(keyword)
    df2 = read_naver_kin(keyword)
    df3 = read_naver_cafe(keyword)
    df4 = read_daum_news(keyword)
    df5 = read_bigkinds(keyword)
    df6 = read_tweeter(keyword)
    df7 = read_instagram(keyword)
    df= df1.append([df2,df3,df4,df5,df6,df7])
    
    return df

In [ ]:
df = readall(keyword)

In [ ]:
df.head()

,title,content,date
0,부산 가족외식 하기 좋았던 소갈비 맛집,가족외식을 하고 돌아왔답니다~^^ 우리 식구들은 입맛이 정말 가지각색이라서 해산물파...,2020-05-29
1,색달랐던 울산 가족외식,#울산가족외식 #대게명가 번호 :052-281-7802 주소 : 울산 북구 정자1길...,2020-06-05
2,잘 하고 온 부산가족외식,부산가족외식을 하고 왔는데 진짜 마음에 들었어요 서면은 주로 친구들이랑 같이 갔었지...,2020-03-10
3,주례 외식&회식을 해결,"부담없이 즐길수 있는 셀프바! 그리고 소, 돼지, 오리까지 다양한 메뉴가 있는 곳이...",2020-06-05
4,갈매역 맛집 정샤브 가성비갑 외식장소,"가족외식 하면 어찌어찌 하면 비용 금방 올라가는데 샤브샤브에 월남쌈 무한리필에, 차...",2020-06-14


In [ ]:
temp_list = list()
for index, data in enumerate(df['date']):
    try:
        pd.to_datetime(data)
    except:
        temp_list.append(index)

temp_list

In [ ]:
%%writefile hanpre/stopwords.py

stopwords = """아휴
식
이
기
고
가
인
리
지
아이구
아이쿠
아이고
어
나
우리
저희
따라
의해
을
를
에
의
가
으로
로
에게
뿐이다
의거하여
근거하여
입각하여
기준으로
예하면
예를 들면
예를 들자면
저
소인
소생
저희
지말고
하지마
하지마라
다른
물론
또한
그리고
비길수 없다
해서는 안된다
뿐만 아니라
만이 아니다
만은 아니다
막론하고
관계없이
그치지 않다
그러나
그런데
하지만
든간에
논하지 않다
따지지 않다
설사
비록
더라도
아니면
만 못하다
하는 편이 낫다
불문하고
향하여
향해서
향하다
쪽으로
틈타
이용하여
타다
오르다
제외하고
이 외에
이 밖에
하여야
비로소
한다면 몰라도
외에도
이곳
여기
부터
기점으로
따라서
할 생각이다
하려고하다
이리하여
그리하여
그렇게 함으로써
하지만
일때
할때
앞에서
중에서
보는데서
으로써
로써
까지
해야한다
일것이다
반드시
할줄알다
할수있다
할수있어
임에 틀림없다
한다면
등
등등
제
겨우
단지
다만
할뿐
딩동
댕그
대해서
대하여
대하면
훨씬
얼마나
얼마만큼
얼마큼
남짓
여
얼마간
약간
다소
좀
조금
다수
몇
얼마
지만
하물며
또한
그러나
그렇지만
하지만
이외에도
대해 말하자면
뿐이다
다음에
반대로
반대로 말하자면
이와 반대로
바꾸어서 말하면
바꾸어서 한다면
만약
그렇지않으면
까악
툭
딱
삐걱거리다
보드득
비걱거리다
꽈당
응당
해야한다
에 가서
각
각각
여러분
각종
각자
제각기
하도록하다
와
과
그러므로
그래서
고로
한 까닭에
하기 때문에
거니와
이지만
대하여
관하여
관한
과연
실로
아니나다를가
생각한대로
진짜로
한적이있다
하곤하였다
하
하하
허허
아하
거바
와
오
왜
어째서
무엇때문에
어찌
하겠는가
무슨
어디
어느곳
더군다나
하물며
더욱이는
어느때
언제
야
이봐
어이
여보시오
흐흐
흥
휴
헉헉
헐떡헐떡
영차
여차
어기여차
끙끙
아야
앗
아야
콸콸
졸졸
좍좍
뚝뚝
주룩주룩
솨
우르르
그래도
또
그리고
바꾸어말하면
바꾸어말하자면
혹은
혹시
답다
및
그에 따르는
때가 되어
즉
지든지
설령
가령
하더라도
할지라도
일지라도
지든지
몇
거의
하마터면
인젠
이젠
된바에야
된이상
만큼	어찌됏든
그위에
게다가
점에서 보아
비추어 보아
고려하면
하게될것이다
일것이다
비교적
좀
보다더
비하면
시키다
하게하다
할만하다
의해서
연이서
이어서
잇따라
뒤따라
뒤이어
결국
의지하여
기대여
통하여
자마자
더욱더
불구하고
얼마든지
마음대로
주저하지 않고
곧
즉시
바로
당장
하자마자
밖에 안된다
하면된다
그래
그렇지
요컨대
다시 말하자면
바꿔 말하면
즉
구체적으로
말하자면
시작하여
시초에
이상
허
헉
허걱
바와같이
해도좋다
해도된다
게다가
더구나
하물며
와르르
팍
퍽
펄렁
동안
이래
하고있었다
이었다
에서
로부터
까지
예하면
했어요
해요
함께
같이
더불어
마저
마저도
양자
모두
습니다
가까스로
하려고하다
즈음하여
다른
다른 방면으로
해봐요
습니까
했어요
말할것도 없고
무릎쓰고
개의치않고
하는것만 못하다
하는것이 낫다
매
매번
들
모
어느것
어느
로써
갖고말하자면
어디
어느쪽
어느것
어느해
어느 년도
라 해도
언젠가
어떤것
어느것
저기
저쪽
저것
그때
그럼
그러면
요만한걸
그래
그때
저것만큼
그저
이르기까지
할 줄 안다
할 힘이 있다
너
너희
당신
어찌
설마
차라리
할지언정
할지라도
할망정
할지언정
구토하다
게우다
토하다
메쓰겁다
옆사람
퉤
쳇
의거하여
근거하여
의해
따라
힘입어
그
다음
버금
두번째로
기타
첫번째로
나머지는
그중에서
견지에서
형식으로 쓰여
입장에서
위해서
단지
의해되다
하도록시키다
뿐만아니라
반대로
전후
전자
앞의것
잠시
잠깐
하면서
그렇지만
다음에
그러한즉
그런즉
남들
아무거나
어찌하든지
같다
비슷하다
예컨대
이럴정도로
어떻게
만약
만일
위에서 서술한바와같이
인 듯하다
하지 않는다면
만약에
무엇
무슨
어느
어떤
아래윗
조차
한데
그럼에도 불구하고
여전히
심지어
까지도
조차도
하지 않도록
않기 위하여
때
시각
무렵
시간
동안
어때
어떠한
하여금
네
예
우선
누구
누가 알겠는가
아무도
줄은모른다
줄은 몰랏다
하는 김에
겸사겸사
하는바
그런 까닭에
한 이유는
그러니
그러니까
때문에
그
너희
그들
너희들
타인
것
것들
너
위하여
공동으로
동시에
하기 위하여
어찌하여
무엇때문에
붕붕
윙윙
나
우리
엉엉
휘익
윙윙
오호
아하
어쨋든
만 못하다	하기보다는
차라리
하는 편이 낫다
흐흐
놀라다
상대적으로 말하자면
마치
아니라면
쉿
그렇지 않으면
그렇지 않다면
안 그러면
아니었다면
하든지
아니면
이라면
좋아
알았어
하는것도
그만이다
어쩔수 없다
하나
일
일반적으로
일단
한켠으로는
오자마자
이렇게되면
이와같다면
전부
한마디
한항목
근거로
하기에
아울러
하지 않도록
않기 위해서
이르기까지
이 되다
로 인하여
까닭으로
이유만으로
이로 인하여
그래서
이 때문에
그러므로
그런 까닭에
알 수 있다
결론을 낼 수 있다
으로 인하여
있다
어떤것
관계가 있다
관련이 있다
연관되다
어떤것들
에 대해
이리하여
그리하여
여부
하기보다는
하느니
하면 할수록
운운
이러이러하다
하구나
하도다
다시말하면
다음으로
에 있다
에 달려 있다
우리
우리들
오히려
하기는한데
어떻게
어떻해
어찌됏어
어때
어째서
본대로
자
이
이쪽
여기
이것
이번
이렇게말하자면
이런
이러한
이와 같은
요만큼
요만한 것
얼마 안 되는 것
이만큼
이 정도의
이렇게 많은 것
이와 같다
이때
이렇구나
것과 같이
끼익
삐걱
따위
와 같은 사람들
부류의 사람들
왜냐하면
중의하나
오직
오로지
에 한하다
하기만 하면
도착하다
까지 미치다
도달하다
정도에 이르다
할 지경이다
결과에 이르다
관해서는
여러분
하고 있다
한 후
혼자
자기
자기집
자신
우에 종합한것과같이
총적으로 보면
총적으로 말하면
총적으로
대로 하다
으로서
참
그만이다
할 따름이다
쿵
탕탕
쾅쾅
둥둥
봐
봐라
아이야
아니
와아
응
아이
참나
년
월
일
령
영
일
이
삼
사
오
육
륙
칠
팔
구
이천육
이천칠
이천팔
이천구
하나
둘
셋
넷
다섯
여섯
일곱
여덟
아홉
령
영"""

stopwords = stopwords.replace("\n",",").split(',')

stopwords_kr = stopwords + [
'하지만', '그리고', '그런데', '저는','제가','있습니다', '않습니다', 'ㅋㅋ', 'ㅋㅋㅋ', 'ㅎㅎ', 'ㅎㅎㅎ',
'그럼', '이런', '저런', '합니다', '있어요', '참고로', '그러고', '아시죠', '하는',
'있어서', '그냥', '같아요', '입니다', 'com', '아주', '않습니다',
'같은', '해서', '있고', '않고', '없는', '있는데',
'www', '먼저', '다시', '있도록', '등을','대한','있으니','미리','것이','모든', '00','없이',
'정도','오늘', '근데', '같습니다','통해', '내가','나는', '나오는', 'http', '바랍니다',
'있어', '위한','요즘','한번','밝혔다','계속', '됩니다','사실', '더욱', '하기',
'아닌', '쉬게', '여러', '가능합니다', '보고', '분들이', '있다는', '라는', '등의',
'수도', '되었습니다', '부탁드립니다', '있는데요', '되고', '있게', '이를', '통한', '것으로', '있으며', '가지', 
'경우', '000원','이미','가서','지금','by','그런', '보니','on', '했는데', '전에','좋을',
'위해', '만원', '인분', '인당', '배가', '완전', '자주', '양이', '오늘은', '먹었어요', '처음', '되는', '좋아요',
'먹은', '것은', '먹으니', '높은', '정도로', '넣고', 'ㅠㅠ', '나왔어요', 'ㅋㅋㅋㅋ', '아직',
'가는', '것도', '것을','있다고', '기자', '따른','보면','알고','있을','11','10','19',
'주는', '되면', '매우', '보다', '대신', '때문이다', '가진', '이는', '출처', '우리는', '앞으로', 
'했다', '곳이', '한다', '이렇게', '하고', '된다', '매우', '의한', '설명하시오', '아래와', '이후', '교시', '들어', '있지만',
'것이다', '출처', '나의', '말합니다', '나를', '된다', '인해', '가능한', '관련', '하지', '없다', '비건', 
'image', '이제', '라고','있었어요','해도','좋습니다', '없고','위에','나온', '않는', '가지고','않은','그렇게','하는데',
'그런지','많아서','것입니다','이건','또는', '있었습니다', '되어', '없습니다', '싶은',
'이거', '사용할', '전혀', '아예', '표시되지', '살짝', '오는', '다들', '밖에', '요즘은', 
'개의', 'ㅋㅋㅋㅋㅋ', '안에', '이상의', '생각합니다', '넘어갑니다더싸다특가', '등으로', '가면', '파는',
'하네요', '넣어', '하니', '하네요', '사용한', '개의', '년까지', '했습니다', '하게', '하니', '하네요', '있기', '있으면', 
'아니다', '같아서', '먹일', '것이라는', '보인다', '사상', '만명', 
'않는다', '있다면', '하는데요', '하니', '하네요', '일부터', '뭔가', '와의', '것이라', '아닐까', '주고',
'것에', '것이라고', '그것을', '누구나', '두고', '싶습니다', '중에', '못하는', '것이죠', '한다는', '그렇다면', 
'봅니다', '수밖에', '싶네요', '맛있어요', '맛있었어요', '올려서', '오랜만에', '주문했어요', '우리가', '사람이',
'그의', '헤헤헤', '기반', '그는', '이라는', '이다', '말했다', '그들의', '주로', '이번에', '아래', '만큼', '억원',
'받고', '해야', '있죠', '받을', '오전', '오후', '나타났다', '반면', '뜻하는', '대상으로', '관계자는', 
'갖고', '다음과', '달러', '먹으면', '아니고', '있음', '예상', '경우가', '원하는', '뜻하는', '이제는', '년간', 
'와서', '이게', '있었다', '지난', '사용하는', '번째', '등이', '이하', '인한', '이에', '서비스입니다', '있는', '그대로', 
'있답니다', '일까지', '위치한', '대해', '번째', '먹고', '먹기', '먹는', '먹을', '마시고', '먹어도', '먹었는데', '먹어야', '먹었던',
'마시는', '마시면', '가장', '엄청', '살짝', '같은데', '넘나', '', '어서', '어요', '면서', '다가', '래서', '이나', '거나', '어도', '더니', 
'거르', '도록', '드릴', '이면', '이지', '어제', '지나', '서도', '이기', '차려', '지고', '별로', '두기', '거기', '는걸', '있다.',
'이날', '했다.', '나타났다.', '발표했다.', '나섰다.',
'확실히', '등은', '지난달', '대부분', '이쯤', '그전', '가고', '어떠', '나아', '사야', '이내', '이외', '가만', '몰아', '했지', '밝혔다.',
'하면', 'photo','아니라','역시','저희는', '저도','특히','30', '따로', '안녕하세요','항상','현재',
'많은', '많이', '정말', '너무', 'https', 'Coupang', 'Coupa', 'ng'
] 



Overwriting hanpre/stopwords.py
